### Compare performance of Arrow, CSV and Parquet

In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import numpy as np
import os
import psutil

##### 1. Load and prepare data

In [11]:
# Read covid data from github
df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316650,ZWE,Africa,Zimbabwe,2023-06-03,265139.0,32.0,21.000,5695.0,0.0,0.429,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316651,ZWE,Africa,Zimbabwe,2023-06-04,265139.0,0.0,16.571,5695.0,0.0,0.286,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316652,ZWE,Africa,Zimbabwe,2023-06-05,265139.0,0.0,15.571,5695.0,0.0,0.000,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316653,ZWE,Africa,Zimbabwe,2023-06-06,265139.0,0.0,15.571,5695.0,0.0,0.000,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN


In [91]:
df_copy = df.copy()
df_copy

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316650,ZWE,Africa,Zimbabwe,2023-06-03,265139.0,32.0,21.000,5695.0,0.0,0.429,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316651,ZWE,Africa,Zimbabwe,2023-06-04,265139.0,0.0,16.571,5695.0,0.0,0.286,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316652,ZWE,Africa,Zimbabwe,2023-06-05,265139.0,0.0,15.571,5695.0,0.0,0.000,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316653,ZWE,Africa,Zimbabwe,2023-06-06,265139.0,0.0,15.571,5695.0,0.0,0.000,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN


In [77]:
# show all object columns
df_copy.select_dtypes(include=['object']).columns

Index(['iso_code', 'continent', 'location', 'date', 'tests_units'], dtype='object')

In [78]:
# convert object columns to category
for col in df_copy.select_dtypes(include=['object']).columns:
    df_copy[col] = df_copy[col].astype('category')

# show all object columns
df_copy.select_dtypes(include=['object']).columns

Index([], dtype='object')

In [79]:
# show all category columns
df_copy.select_dtypes(include=['category']).columns

Index(['iso_code', 'continent', 'location', 'date', 'tests_units'], dtype='object')

In [80]:
# null values in category columns
df_copy.select_dtypes(include=['category']).isnull().sum()

iso_code            0
continent       15042
location            0
date                0
tests_units    209867
dtype: int64

In [81]:
# sample category column with null values
df_copy['continent'].sample(10)

152535             Asia
256175    North America
98341           Oceania
294580           Europe
7906             Africa
234002           Africa
183503           Europe
130615             Asia
236399           Africa
240237    North America
Name: continent, dtype: category
Categories (6, object): ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']

In [82]:
# sample rows with null values
df_copy[df_copy['continent'].isnull()].sample(10)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
89329,OWID_EUN,NaN,European Union,2021-05-24,31781571.0,29401.0,47711.857,728850.0,467.0,905.571,...,NaN,NaN,NaN,NaN,NaN,4.501468e+08,NaN,NaN,NaN,NaN
263896,OWID_SAM,NaN,South America,2022-08-05,62446247.0,61313.0,67079.571,1316966.0,364.0,355.571,...,NaN,NaN,NaN,NaN,NaN,4.368167e+08,NaN,NaN,NaN,NaN
211136,OWID_OCE,NaN,Oceania,2021-12-10,398906.0,124.0,1758.000,4768.0,6.0,12.714,...,NaN,NaN,NaN,NaN,NaN,4.503886e+07,NaN,NaN,NaN,NaN
15137,OWID_ASI,NaN,Asia,2020-04-20,382597.0,12547.0,11936.857,14690.0,327.0,528.714,...,NaN,NaN,NaN,NaN,NaN,4.721383e+09,NaN,NaN,NaN,NaN
15311,OWID_ASI,NaN,Asia,2020-10-11,12292304.0,112048.0,114674.571,212351.0,1617.0,1690.000,...,NaN,NaN,NaN,NaN,NaN,4.721383e+09,NaN,NaN,NaN,NaN
203344,OWID_NAM,NaN,North America,2022-11-30,117023646.0,5749.0,50517.571,1533827.0,21.0,438.143,...,NaN,NaN,NaN,NaN,NaN,6.003237e+08,NaN,NaN,NaN,NaN
1293,OWID_AFR,NaN,Africa,2020-02-13,NaN,0.0,0.000,NaN,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN,NaN
301248,OWID_UMC,NaN,Upper middle income,2022-02-28,114646198.0,447462.0,427542.714,2369468.0,3456.0,2898.000,...,NaN,NaN,NaN,NaN,NaN,2.525921e+09,NaN,NaN,NaN,NaN
120355,OWID_HIC,NaN,High income,2020-08-17,8934318.0,75361.0,78404.571,404674.0,1278.0,1402.143,...,NaN,NaN,NaN,NaN,NaN,1.250515e+09,NaN,NaN,NaN,NaN
120891,OWID_HIC,NaN,High income,2022-02-04,207518882.0,1947203.0,1976112.000,2194594.0,6218.0,5332.857,...,NaN,NaN,NaN,NaN,NaN,1.250515e+09,NaN,NaN,NaN,NaN


In [30]:
# select distinct values from the location column
df_copy['location'].unique()

['Afghanistan', 'Africa', 'Albania', 'Algeria', 'American Samoa', ..., 'Western Sahara', 'World', 'Yemen', 'Zambia', 'Zimbabwe']
Length: 255
Categories (255, object): ['Afghanistan', 'Africa', 'Albania', 'Algeria', ..., 'World', 'Yemen', 'Zambia', 'Zimbabwe']

In [31]:
# select distinct values from the continent column
df_copy['continent'].unique()

['Asia', NaN, 'Europe', 'Africa', 'Oceania', 'North America', 'South America']
Categories (6, object): ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']

In [32]:
# select distinct values from the tests_units column
df_copy['tests_units'].unique()

[NaN, 'tests performed', 'units unclear', 'samples tested', 'people tested']
Categories (4, object): ['people tested', 'samples tested', 'tests performed', 'units unclear']

In [92]:
# drop test_units and continent columns
df_copy.drop(['tests_units', 'continent'], axis=1, inplace=True)

In [93]:
df_copy

,iso_code,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316650,ZWE,Zimbabwe,2023-06-03,265139.0,32.0,21.000,5695.0,0.0,0.429,16245.726,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316651,ZWE,Zimbabwe,2023-06-04,265139.0,0.0,16.571,5695.0,0.0,0.286,16245.726,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316652,ZWE,Zimbabwe,2023-06-05,265139.0,0.0,15.571,5695.0,0.0,0.000,16245.726,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316653,ZWE,Zimbabwe,2023-06-06,265139.0,0.0,15.571,5695.0,0.0,0.000,16245.726,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN


In [94]:
# convert object columns to category
for col in df_copy.select_dtypes(include=['object']).columns:
    df_copy[col] = df_copy[col].astype('category')

# show all object columns
df_copy.select_dtypes(include=['object']).columns

# show all category columns


Index([], dtype='object')

In [95]:
# show data types
df_copy.dtypes


iso_code                                   category
location                                   category
date                                       category
total_cases                                 float64
new_cases                                   float64
                                             ...   
population                                  float64
excess_mortality_cumulative_absolute        float64
excess_mortality_cumulative                 float64
excess_mortality                            float64
excess_mortality_cumulative_per_million     float64
Length: 65, dtype: object

In [96]:
# increase dataset to 1 million rows and reset index
df_copy = df_copy.sample(n=1000000, replace=True).reset_index(drop=True)
df_copy

,iso_code,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,FRA,France,2021-02-24,3532628.0,20736.0,18225.143,84595.0,428.0,319.714,54662.116,...,35.6,NaN,5.98,82.66,0.901,67813000.0,NaN,NaN,NaN,NaN
1,TON,Tonga,2020-06-25,NaN,0.0,0.000,NaN,0.0,0.000,NaN,...,44.4,NaN,2.60,70.91,0.725,106867.0,NaN,NaN,NaN,NaN
2,BTN,Bhutan,2020-12-19,443.0,1.0,1.000,NaN,0.0,0.000,566.165,...,NaN,79.807,1.70,71.78,0.654,782457.0,NaN,NaN,NaN,NaN
3,FRA,France,2021-10-20,6875557.0,5689.0,4431.571,114957.0,36.0,25.000,106388.924,...,35.6,NaN,5.98,82.66,0.901,67813000.0,NaN,NaN,NaN,NaN
4,OWID_EUR,Europe,2021-04-20,42982232.0,122535.0,161075.714,1020925.0,3592.0,3447.857,57709.159,...,NaN,NaN,NaN,NaN,NaN,744807803.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,BRN,Brunei,2022-08-03,215283.0,0.0,1303.286,122.0,0.0,0.000,479470.025,...,30.9,NaN,2.70,75.86,0.838,449002.0,NaN,NaN,NaN,NaN
999996,PRK,North Korea,2020-07-17,NaN,0.0,0.000,NaN,0.0,0.000,NaN,...,NaN,NaN,13.20,72.27,NaN,26069416.0,NaN,NaN,NaN,NaN
999997,LUX,Luxembourg,2021-12-25,98983.0,543.0,410.429,907.0,0.0,1.143,152845.657,...,26.0,NaN,4.51,82.25,0.916,647601.0,NaN,NaN,NaN,NaN
999998,PRY,Paraguay,2020-11-06,65258.0,630.0,566.857,1454.0,13.0,13.571,9624.016,...,21.6,79.602,1.30,74.25,0.728,6780745.0,NaN,NaN,NaN,NaN


In [89]:
1000000000 >> 20

953

In [90]:
2 ** 20

1048576

In [40]:
# update sample number column with index
df_copy["Sample Number"] = df_copy.index
df_copy

,iso_code,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,Sample Number
0,TCA,Turks and Caicos Islands,2022-01-07,3562.0,0.0,39.857,26.0,0.0,0.000,77898.788,...,NaN,NaN,80.22,NaN,45726.0,NaN,NaN,NaN,NaN,0
1,JPN,Japan,2020-12-07,162067.0,1969.0,2186.714,2335.0,20.0,30.857,1307.501,...,NaN,13.05,84.63,0.919,123951696.0,NaN,NaN,NaN,NaN,1
2,PHL,Philippines,2021-09-06,2103296.0,22380.0,18180.857,34337.0,103.0,143.857,18201.056,...,78.463,1.00,71.23,0.718,115559008.0,NaN,NaN,NaN,NaN,2
3,CAN,Canada,2022-11-13,4380999.0,0.0,2627.857,46537.0,0.0,49.143,113927.332,...,NaN,2.50,82.43,0.929,38454328.0,44146.23,5.13,7.94,1148.0172,3
4,BMU,Bermuda,2022-11-11,18428.0,0.0,0.000,149.0,0.0,0.000,287009.205,...,NaN,NaN,82.59,NaN,64207.0,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,NZL,New Zealand,2020-06-17,1156.0,0.0,0.286,22.0,0.0,0.000,222.938,...,NaN,2.61,82.29,0.931,5185289.0,NaN,NaN,NaN,NaN,999995
999996,POL,Poland,2021-01-19,1446149.0,4835.0,6836.714,33777.0,291.0,300.286,36283.307,...,NaN,6.62,78.73,0.880,39857144.0,NaN,NaN,NaN,NaN,999996
999997,MRT,Mauritania,2023-01-21,63435.0,0.0,0.000,997.0,0.0,0.000,13393.802,...,15.950,NaN,64.92,0.546,4736146.0,NaN,NaN,NaN,NaN,999997
999998,PER,Peru,2022-10-03,4145048.0,632.0,596.286,216578.0,3.0,15.000,121735.629,...,NaN,1.60,76.74,0.777,34049588.0,NaN,NaN,NaN,NaN,999998


In [97]:
# show missing values in dataset
df_copy.isnull().sum()


iso_code                                        0
location                                        0
date                                            0
total_cases                                114102
new_cases                                   28006
                                            ...  
population                                      0
excess_mortality_cumulative_absolute       965208
excess_mortality_cumulative                965208
excess_mortality                           965208
excess_mortality_cumulative_per_million    965208
Length: 65, dtype: int64

In [98]:
# fill missing values with 0 for all non-category columns
df_nonan = df_copy.copy()
for col in df_nonan.select_dtypes(exclude=['category']).columns:
    df_nonan[col] = df_nonan[col].fillna(0)

In [99]:
# show missing values in dataset
df_nonan.isnull().sum()

iso_code                                   0
location                                   0
date                                       0
total_cases                                0
new_cases                                  0
                                          ..
population                                 0
excess_mortality_cumulative_absolute       0
excess_mortality_cumulative                0
excess_mortality                           0
excess_mortality_cumulative_per_million    0
Length: 65, dtype: int64

In [100]:
df_nonan

,iso_code,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,FRA,France,2021-02-24,3532628.0,20736.0,18225.143,84595.0,428.0,319.714,54662.116,...,35.6,0.000,5.98,82.66,0.901,67813000.0,0.0,0.0,0.0,0.0
1,TON,Tonga,2020-06-25,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,44.4,0.000,2.60,70.91,0.725,106867.0,0.0,0.0,0.0,0.0
2,BTN,Bhutan,2020-12-19,443.0,1.0,1.000,0.0,0.0,0.000,566.165,...,0.0,79.807,1.70,71.78,0.654,782457.0,0.0,0.0,0.0,0.0
3,FRA,France,2021-10-20,6875557.0,5689.0,4431.571,114957.0,36.0,25.000,106388.924,...,35.6,0.000,5.98,82.66,0.901,67813000.0,0.0,0.0,0.0,0.0
4,OWID_EUR,Europe,2021-04-20,42982232.0,122535.0,161075.714,1020925.0,3592.0,3447.857,57709.159,...,0.0,0.000,0.00,0.00,0.000,744807803.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,BRN,Brunei,2022-08-03,215283.0,0.0,1303.286,122.0,0.0,0.000,479470.025,...,30.9,0.000,2.70,75.86,0.838,449002.0,0.0,0.0,0.0,0.0
999996,PRK,North Korea,2020-07-17,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,0.0,0.000,13.20,72.27,0.000,26069416.0,0.0,0.0,0.0,0.0
999997,LUX,Luxembourg,2021-12-25,98983.0,543.0,410.429,907.0,0.0,1.143,152845.657,...,26.0,0.000,4.51,82.25,0.916,647601.0,0.0,0.0,0.0,0.0
999998,PRY,Paraguay,2020-11-06,65258.0,630.0,566.857,1454.0,13.0,13.571,9624.016,...,21.6,79.602,1.30,74.25,0.728,6780745.0,0.0,0.0,0.0,0.0


#### 1.2 Write to disk

In [101]:
# write to csv
df_copy.to_csv("../data/covid-data.csv")

In [102]:
# write to parquet
df_copy.to_parquet("../data/covid-data.parquet")

In [103]:
# write with nan values to arrow
# convert from pandas to arrow
table = pa.Table.from_pandas(df_copy)
# write to arrow file
with pa.OSFile('../data/covid-data-redo.arrow', 'wb') as sink:
    with pa.RecordBatchFileWriter(sink, table.schema) as writer:
        writer.write_table(table)

In [104]:
# write with nonan values to arrow file
# convert from pandas to arrow
table = pa.Table.from_pandas(df_nonan)
# write to arrow file
with pa.OSFile('../data/covid-data-redo-nonan.arrow', 'wb') as sink:
    with pa.RecordBatchFileWriter(sink, table.schema) as writer:
        writer.write_table(table)

In [48]:
# write to feather
# convert from pandas to arrow
table = pa.Table.from_pandas(df_copy)
# write to feather file
pq.write_table(table, '../data/covid-data.feather')

In [106]:
%%timeit
# read csv file and calculate mean of new_cases column
df_csv = pd.read_csv("../data/covid-data.csv")["new_cases"].mean()

2.81 s ± 36.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
%%timeit
# read parquet file and calculate mean of new_cases column
df_parquet = pd.read_parquet("../data/covid-data.parquet")["new_cases"].mean()

188 ms ± 3.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
import time, timeit

In [59]:
# read parquet file and calculate mean of new_cases column
start = time.time()
df_parquet = pd.read_parquet("../data/covid-data.parquet", columns=["new_cases"]).mean()
end = time.time()
print("Parquet: ", end - start)

Parquet:  0.018027544021606445


In [108]:
%%timeit
df_parquet = pd.read_parquet("../data/covid-data.parquet", columns=["new_cases"]).mean()

20.9 ms ± 307 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Read Arrow file using file API

In [109]:
%%timeit
# Read arrow file using file api and calculate mean of new_cases column
with pa.OSFile('../data/covid-data-redo.arrow', 'rb') as f:
    df_arrow = pa.ipc.open_file(f).read_all().column("new_cases")

result = df_arrow.to_pandas().mean()

180 ms ± 718 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Read Arrow file using memory-mapped API with missing values

In [110]:
%%timeit
# Read arrow file using memory map and calculate mean of new_cases column
source = pa.memory_map("../data/covid-data-redo.arrow", mode="r")
table = pa.ipc.RecordBatchFileReader(source).read_all().column("new_cases")
df_arrow = table.to_pandas().mean()

46.9 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Read Arrow file using memory-mapped API without missing values

In [111]:
%%timeit
# Read arrow file using memory map and calculate mean of new_cases column
source = pa.memory_map("../data/covid-data-redo-nonan.arrow", mode="r")
table = pa.ipc.RecordBatchFileReader(source).read_all().column("new_cases")
df_arrow = table.to_pandas().mean()

34 ms ± 899 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
